Options for encoding ordinal response

- Naive categorical encoding (ignore order)
- Integer encoding (really regression not classification)
- [Ordinal crossentropy loss for Keras](https://github.com/JHart96/keras_ordinal_categorical_crossentropy)
- [Ordinal regression for TF](https://github.com/gspell/TF-OrdinalRegression)
- "Cumulative" encoding [Cheng et al.]
- Split into K-1 binary classification problems [Frank and Hall] 
      - not sure if it's efficient with neural nets
      - Cheng et al.'s encoding does this in some sense, within a single network

In [1]:
# set up
# if installed, keras uses tf as backend
import keras
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Import ordinal crossentropy loss function
import sys
sys.path.insert(0, "./keras_ordinal_categorical_crossentropy")
import ordinal_categorical_crossentropy as OCC

Using TensorFlow backend.


Example from Keras docs
https://keras.io/getting-started/sequential-model-guide/#examples

In [2]:
# Generate dummy data
x_train = np.random.random((1000, 20))
y_int_train = np.random.randint(10, size=(1000, 1))
y_train = keras.utils.to_categorical(y_int_train, num_classes=10)
x_test = np.random.random((100, 20))
y_int_test = np.random.randint(10, size=(100, 1))
y_test = keras.utils.to_categorical(y_int_test, num_classes=10)

# define classification model
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# set SGD as optimizer
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [3]:
# naive multi-class classification
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 688us/step - loss: 2.4004 - accuracy: 0.0990
Epoch 2/20
1000/1000 [==============================] - 0s 55us/step - loss: 2.3764 - accuracy: 0.0890
Epoch 3/20
1000/1000 [==============================] - 0s 238us/step - loss: 2.3475 - accuracy: 0.0900
Epoch 4/20
1000/1000 [==============================] - 0s 48us/step - loss: 2.3406 - accuracy: 0.1070
Epoch 5/20
1000/1000 [==============================] - 0s 50us/step - loss: 2.3230 - accuracy: 0.1000
Epoch 6/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.3159 - accuracy: 0.1050
Epoch 7/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.3195 - accuracy: 0.0970
Epoch 8/20
1000/1000 [==============================] - 0s 49us/step - loss: 2.3101 - accuracy: 0.0970
Epoch 9/20
1000/1000 [==============================] - 0s 51us/step - loss: 2.3103 - accuracy: 0.0860
Epoch 10/20
1000/1000 [==============================] - 0s 58us/step -

[2.3121721744537354, 0.07000000029802322]

In [4]:
# Classification with ordinal crossentropy loss
model.compile(loss=OCC.loss,
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=20, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 754us/step - loss: 3.0047 - accuracy: 0.1130
Epoch 2/20
1000/1000 [==============================] - 0s 77us/step - loss: 3.0150 - accuracy: 0.1220
Epoch 3/20
1000/1000 [==============================] - 0s 55us/step - loss: 3.0262 - accuracy: 0.1260
Epoch 4/20
1000/1000 [==============================] - 0s 54us/step - loss: 3.0381 - accuracy: 0.1170
Epoch 5/20
1000/1000 [==============================] - 0s 54us/step - loss: 3.0487 - accuracy: 0.1190
Epoch 6/20
1000/1000 [==============================] - 0s 52us/step - loss: 3.0966 - accuracy: 0.1250
Epoch 7/20
1000/1000 [==============================] - 0s 52us/step - loss: 3.0845 - accuracy: 0.1160
Epoch 8/20
1000/1000 [==============================] - 0s 52us/step - loss: 3.0875 - accuracy: 0.1240
Epoch 9/20
1000/1000 [==============================] - 0s 51us/step - loss: 3.0961 - accuracy: 0.1100
Epoch 10/20
1000/1000 [==============================] - 0s 52us/step - 

[3.024740695953369, 0.09000000357627869]

In [5]:
# Integer encoding
model_int = Sequential()
model_int.add(Dense(64, activation='relu', input_dim=20))
model_int.add(Dropout(0.5))
model_int.add(Dense(64, activation='relu'))
model_int.add(Dropout(0.5))
model_int.add(Dense(1, activation='relu'))

model_int.compile(
    ## or try sparse_categorical_crossentropy for integer targets
    loss='mean_squared_error',
    optimizer=sgd,
    metrics=['accuracy', 'mse'])

model_int.fit(x_train, y_int_train, epochs=20, batch_size=128)
score = model_int.evaluate(x_test, y_int_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 854us/step - loss: 15.2446 - accuracy: 0.0900 - mse: 15.2446
Epoch 2/20
1000/1000 [==============================] - 0s 45us/step - loss: 8.9296 - accuracy: 0.1100 - mse: 8.9296
Epoch 3/20
1000/1000 [==============================] - 0s 66us/step - loss: 8.3448 - accuracy: 0.1120 - mse: 8.3448
Epoch 4/20
1000/1000 [==============================] - 0s 59us/step - loss: 8.0190 - accuracy: 0.1170 - mse: 8.0190
Epoch 5/20
1000/1000 [==============================] - 0s 59us/step - loss: 7.9896 - accuracy: 0.0980 - mse: 7.9896
Epoch 6/20
1000/1000 [==============================] - 0s 59us/step - loss: 7.8816 - accuracy: 0.1100 - mse: 7.8816
Epoch 7/20
1000/1000 [==============================] - 0s 242us/step - loss: 7.8467 - accuracy: 0.1090 - mse: 7.8467
Epoch 8/20
1000/1000 [==============================] - 0s 45us/step - loss: 7.8214 - accuracy: 0.1160 - mse: 7.8214
Epoch 9/20
1000/1000 [==============================] - 0s 4

[8.508237838745117, 0.10000000149011612, 8.508237838745117]

In [7]:
# let's try it again but with sparse cross-entropy loss
# Integer encoding
model_sparse = Sequential()
model_sparse.add(Dense(64, activation='relu', input_dim=20))
model_sparse.add(Dropout(0.5))
model_sparse.add(Dense(64, activation='relu'))
model_sparse.add(Dropout(0.5))
model_sparse.add(Dense(10, activation='relu'))

model_sparse.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=sgd,
    metrics=['accuracy'])

model_sparse.fit(x_train, y_int_train, epochs=20, batch_size=128)
score = model_sparse.evaluate(x_test, y_int_test, batch_size=128)
score

Epoch 1/20
1000/1000 [==============================] - 1s 737us/step - loss: 9.2361 - accuracy: 0.1110
Epoch 2/20
1000/1000 [==============================] - 0s 54us/step - loss: 8.2345 - accuracy: 0.0950
Epoch 3/20
1000/1000 [==============================] - 0s 52us/step - loss: 7.9862 - accuracy: 0.0950
Epoch 4/20
1000/1000 [==============================] - 0s 54us/step - loss: 8.0912 - accuracy: 0.0940
Epoch 5/20
1000/1000 [==============================] - 0s 53us/step - loss: 8.4529 - accuracy: 0.0940
Epoch 6/20
1000/1000 [==============================] - 0s 53us/step - loss: 8.9290 - accuracy: 0.0940
Epoch 7/20
1000/1000 [==============================] - 0s 51us/step - loss: 9.1862 - accuracy: 0.0940
Epoch 8/20
1000/1000 [==============================] - 0s 53us/step - loss: 10.1030 - accuracy: 0.0940
Epoch 9/20
1000/1000 [==============================] - 0s 54us/step - loss: 10.9031 - accuracy: 0.0940
Epoch 10/20
1000/1000 [==============================] - 0s 53us/step 

[10.895971298217773, 0.12999999523162842]

Remarks:

- Accuracy is pretty low across all three, but (sparse) integer encoding seems to work well
- Could be that architecture (ie, layers) and optimizer need tuning
- Oh yeah also because features are uncorrelated with response (duh!)

[TODO] Last three options...

## Applying it to our data

Let's apply this to a subset of our data

In [ ]:
parcels = pd.read_csv('../data/toy-parcels.csv')

X = parcels.drop(['recovery'], axis = 1)
X = np.array(X)
Y = parcels['recovery']